In [ ]:
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.decomposition import LatentDirichletAllocation
import nltk
from nltk.cluster import KMeansClusterer
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import json
from nltk.stem import WordNetLemmatizer 
%matplotlib inline

# 1. Read the files

In [ ]:
train = None
with open('training/training_article.txt') as json_file:  
    train = json.load(json_file)

test = None
with open('test/test_article.txt') as json_file:  
    test = json.load(json_file)

# 2. Store the data

## 2.a Store by paragraph

In [ ]:
# train_processed=[]
# for news in train:
#     train_processed.extend(news['body'].split('\n\n'))

In [ ]:
# test_processed=[]
# for news in test:
#     test_processed.extend(news['body'].split('\n\n'))

## 2.b Store by entire articles

In [ ]:
train_processed=[]
for news in train:
    train_processed.append(news['body'])

In [ ]:
test_processed=[]
for news in test:
    test_processed.append(news['body'])

# 3. Text processing

In [ ]:
lemmatizer = WordNetLemmatizer()
def process_text(text):
    words = nltk.word_tokenize(text)
    words = [w.lower() for w in words]
    words = [w.translate(str.maketrans('', '', string.punctuation)) for w in words]
    words = [w for w in words if w not in nltk.corpus.stopwords.words('english')]
    words = [lemmatizer.lemmatize(w) for w in words]
    words = [w for w in words if w.isalnum()] 
    new_text = " ".join(words)
    return new_text

In [ ]:
for i in range(len(train_processed)):
    train_processed[i] = process_text(train_processed[i])
for i in range(len(test_processed)):
    test_processed[i] = process_text(test_processed[i])